In [72]:
import numpy as np
import librosa
import soundfile as sf
import tensorflow as tf
from tensorflow.keras.layers import Input, add, Conv1D, Conv2D, Dense, Flatten, Dropout, MaxPooling2D, Conv1DTranspose, Reshape
import os
import shutil

In [73]:
def load_audio(audio):
    final = []
    segment_length=16384
    for aud in audio:
        file_name = 'C:/Users/anang/OneDrive/Desktop/Image to Audio/audio/cars/'+aud
        audio, _ = librosa.load(file_name, sr=22050)
        audio = (audio - np.min(audio)) / (np.max(audio) - np.min(audio)) * 2 - 1
        print(audio.shape)
        final.append(audio)
        
    return np.array(final)

def preprocess_audio(audio, segment_length=16384):
# Normalize audio
    audio = (audio - np.min(audio)) / (np.max(audio) - np.min(audio)) * 2 - 1

    return np.array(audio)

In [74]:
audio = os.listdir(os.path.join('C:/Users/anang/OneDrive/Desktop/Image to Audio/audio/cars/'))
audio_segments = load_audio(audio)

(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)
(301824,)


In [75]:
audio_segments.shape

(92, 301824)

In [99]:
def build_generator(latent_dim, output_shape):
    model = tf.keras.models.Sequential([
        Input(shape=(100,)),
        Dense(8*(output_shape[0] // 16)),
        Reshape((output_shape[0]//16, 8)),
        Conv1DTranspose(16, kernel_size=25, strides=4, padding='same', activation='relu'),
        Conv1DTranspose(1, kernel_size=25, strides=4, padding='same', activation='tanh'),
    ])

    return model

latent_dim = 100
audio_shape = (301824,1)
generator = build_generator(latent_dim, audio_shape)
generator.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 150912)            15242112  
                                                                 
 reshape_19 (Reshape)        (None, 18864, 8)          0         
                                                                 
 conv1d_transpose_33 (Conv1  (None, 75456, 16)         3216      
 DTranspose)                                                     
                                                                 
 conv1d_transpose_34 (Conv1  (None, 301824, 1)         401       
 DTranspose)                                                     
                                                                 
Total params: 15245729 (58.16 MB)
Trainable params: 15245729 (58.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [100]:
def build_discriminator(shape):
    model = tf.keras.models.Sequential([
        Input(shape=shape),
        Conv1DTranspose(8, kernel_size=25, strides=4, padding='same', activation='leaky_relu'),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model

discriminator = build_discriminator(audio_shape)
discriminator.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_transpose_35 (Conv1  (None, 1207296, 8)        208       
 DTranspose)                                                     
                                                                 
 flatten_6 (Flatten)         (None, 9658368)           0         
                                                                 
 dense_20 (Dense)            (None, 1)                 9658369   
                                                                 
Total params: 9658577 (36.84 MB)
Trainable params: 9658577 (36.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [101]:
discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

In [102]:
discriminator.trainable = False
gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = tf.keras.models.Model(inputs=gan_input, outputs=gan_output)
gan.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy')

In [104]:
def train_wave_gan(generator, discriminator, gan, data, latent_dim, epochs=1200, batch_size=32):
    half_batch = batch_size // 2

    for epoch in range(epochs):

        discriminator.trainable=True

        idx = np.random.randint(0, data.shape[0], half_batch)
        real_samples = data[idx]
        print(real_samples.shape)
        real_labels = np.ones((half_batch, 1))

        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_samples = generator.predict(noise)
        fake_labels = np.zeros((half_batch, 1))

        d_loss_real = discriminator.train_on_batch(real_samples, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_samples, fake_labels)

        d_loss = np.add(d_loss_real, d_loss_fake) * 0.5

        discriminator.trainable = False

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.ones((batch_size, 1))
        
        gan_loss = gan.train_on_batch(noise, valid_y)

        print(f"{epoch} [D loss: {d_loss}] [G loss: {gan_loss}]")

train_wave_gan(generator, discriminator, gan, audio_segments, latent_dim)

(16, 301824)
1/1 [==============================] - 0s 439ms/step
0 [D loss: [5.70521515e-04 1.00000000e+00]] [G loss: 6.805291175842285]
(16, 301824)
1/1 [==============================] - 1s 634ms/step
1 [D loss: [5.54501778e-04 1.00000000e+00]] [G loss: 6.9531073570251465]
(16, 301824)
1/1 [==============================] - 0s 435ms/step
2 [D loss: [5.56986372e-04 1.00000000e+00]] [G loss: 7.172800064086914]
(16, 301824)
1/1 [==============================] - 0s 436ms/step
3 [D loss: [4.94549982e-04 1.00000000e+00]] [G loss: 6.987159729003906]
(16, 301824)
1/1 [==============================] - 1s 514ms/step
4 [D loss: [4.54025867e-04 1.00000000e+00]] [G loss: 7.185739040374756]
(16, 301824)
1/1 [==============================] - 0s 497ms/step
5 [D loss: [4.52807755e-04 1.00000000e+00]] [G loss: 7.141080379486084]
(16, 301824)
1/1 [==============================] - 0s 485ms/step
6 [D loss: [5.17083798e-04 1.00000000e+00]] [G loss: 7.152329444885254]
(16, 301824)
1/1 [===============

In [106]:
def generate_audio(generator, latent_dim, length):
    noise = np.random.normal(0, 1, (1, latent_dim))
    generated_audio = generator.predict(noise)
    return generated_audio.flatten()

generated_audio = generate_audio(generator, latent_dim, audio_shape[0])

# Save the generated audio using soundfile
import soundfile as sf
sf.write('generated_audio.wav', generated_audio, 16000)

1/1 [==============================] - 0s 70ms/step


In [ ]:
"""
audio is loaded and preprocessed from the folders and sent into a np array to create a dataset.
the generator is built. 
    -Using conv1d layers, a dense layer, a reshape layer and an input layer.
    -The conv1d layers are built so the inputs(noise) can be upscaled to reach the size of the required output.
    -The dense layer, does something.
    -The reshape layer does something as well.

the discriminator is built.
    -Using conv1d layers, an input layer, a flatten and a dense layer.
    -The conv1d layer downscaled the input until it reaches a dense(regular nn to make the prediction)
    -the flatten layer makes sure the conv1d outputs are compatible with the dense layer.
    -the dense layer discriminates between real and fake and ouputs either 1 or 0.

the discriminator is compiled using the adam optimizer, the binary crossentropy loss function and accuracy is set as a metric.

the discriminator's weights are frozen to prevent it from training along with the generator in the gan model making it extremely good at spotting real from
fake thereby making the gan model feel bad.
the gan model is built.
    -an input layer is created to accept the noise as input with the specific dimensions(latent space)
    -the discriminator is included as an output to the gan model which takes in the generator as an input and in the end discerns real from fake
    only adjusting the generator's weights in the process without doing the same to the discriminator.
    -the model is compiled using the adam optimizer and a binary crossentropy."""